In [41]:
import pandas as pd
# import pandas_ta as ta
import time
import numpy as np
from datetime import timezone,timedelta,datetime
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nest_asyncio
import yfinance as yf
import quantstats as qs
import mplfinance as mpf
import pickle
from  ib_insync import *
import nest_asyncio

In [16]:
def load_dataframe(file_path):
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
    return df

In [17]:
file_path = 'shared_dataframe.pkl'

while True:
    # Load the updated DataFrame
    df = load_dataframe(file_path)
    
    # Process the DataFrame as needed
    print(df)
    
    # Wait for 5 minutes before fetching again
    time.sleep(300)  # 300 seconds = 5 minutes

                        date    open    high     low   close    volume
0 2024-09-12 16:01:39.436420  468.78  474.04  466.85  468.62  335885.0
1 2024-09-12 16:02:42.464275  468.78  474.04  466.85  468.62  338319.0
2 2024-09-12 16:03:45.490487  468.78  474.04  466.85  468.62  339545.0


KeyboardInterrupt: 

In [18]:
# df1 = pd.read_csv("/content/drive/MyDrive/2015-2024_TQQQ_5min_IBKR.csv")
# df1['date'] = pd.to_datetime(df1['date'])
df.set_index('date', inplace=True)
df.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        }, inplace=True)
df.index = pd.to_datetime(df.index,utc=True)
df.index = df.index.tz_convert('UTC')
df.index = df.index.tz_localize(None)
df.index = df.index.floor('s')
#df = df.between_time('09:30', '16:00')
df

/var/folders/_j/xlsbmg1j0x341kvd3qz81kgr0000gn/T/ipykernel_18756/2603723943.py:14: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df.index = df.index.floor('S')


,open,high,low,close,volume
date,,,,,
2024-09-12 16:01:39,468.78,474.04,466.85,468.62,335885.0
2024-09-12 16:02:42,468.78,474.04,466.85,468.62,338319.0
2024-09-12 16:03:45,468.78,474.04,466.85,468.62,339545.0


In [55]:
for date, day_data in df.groupby(df.index.date):
        day_data = day_data.reset_index()
        print(day_data['date'].dt.time)

0    16:01:39
1    16:02:42
2    16:03:45
Name: date, dtype: object


In [13]:
# Define the inputs
capital = 25000           # Initial account value
leverage = 4              # Leverage multiplier
commission_per_share = 0.0035       # Comission fees and slippage
account_risk=0.01         # Percentage of total account risked per trade
profit_multiplier=10      # Profit target multiplier from entry price
#first_column_index = df.index

In [11]:
def calculate_position_size(entry_price, stop_loss_price):
    risk_per_trade = capital * account_risk
    position_size = risk_per_trade / abs(entry_price - stop_loss_price)
    return position_size
# Example function to place a trade

In [12]:
# Function to calculate ORB strategy
nest_asyncio.apply()
df1 = pd.DataFrame()
ib = IB()
ib.connect('127.0.0.1',7497,clientId=10)
contract = Stock('QQQ','SMART','USD')

def place_trade(entry_price, stop_loss_price, target_price,type):
    position_size = calculate_position_size(entry_price, stop_loss_price)
    if type == 1:
        order = MarketOrder('BUY', position_size)
        trade = ib.placeOrder(contract, order)
        # Set stop loss and profit target
        stop_loss_order = StopOrder('SELL', position_size, stop_loss_price)
        profit_target_order = LimitOrder('SELL', position_size, target_price)
    elif type == -1:
        order = MarketOrder('SELL', position_size)
        trade = ib.placeOrder(contract, order)
        # Set stop loss and profit target
        stop_loss_order = StopOrder('BUY', position_size, stop_loss_price)
        profit_target_order = LimitOrder('BUY', position_size, target_price)
    # Place orders
    ib.placeOrder(contract, stop_loss_order)
    ib.placeOrder(contract, profit_target_order)
    
    return trade

def close_position(position):
              contract = Stock('QQQ','SMART','USD')
              try:
                  contract = position.contract
                  quantity = position.position
                  if quantity > 0:
                      order = MarketOrder('SELL', quantity)
                  else:
                      order = MarketOrder('BUY', -quantity)
                  trade = ib.placeOrder(contract, order)
                  return trade
              except Exception as e:
                  print(f"Error:{e}")
entry_price = None
target_time = pd.to_datetime('15:30:00').time()

def apply_orb_strategy(data, capital, leverage, commission_per_share):
    total_cap=capital
    count = 0
    #initial_hold_quantity = capital/data['Close'][0]
    for date, day_data in data.groupby(data.index.date):
        day_data = day_data.reset_index()
        if day_data.empty or len(day_data) < 2:
            continue  # Skip if there's no data or not enough candles
        # Define the first 5-minute range
        opening_range = day_data.loc[0]
        open_price = opening_range['Open']
        high_price = opening_range['High']
        low_price = opening_range['Low']
        close_price = opening_range['Close']
        if close_price > open_price:# Bullish first candle
          entry_price = day_data.loc[1, 'Open']
          stop_loss = low_price
          direction = 1
        #   buy_points['date'].append(date)
        #   buy_points['price'].append(entry_price)

        elif close_price < open_price: # Bearish first candle
          entry_price = day_data.loc[1, 'Open']
          stop_loss = high_price
          direction = -1
        #   sell_points['date'].append(date)
        #   sell_points['price'].append(entry_price)
        else:
          direction = 0
          continue

        # Calculate risk per trade
        risk_per_trade = account_risk * capital
        # if entry_price == stop_loss:
        #   continue
        shares = risk_per_trade / abs(entry_price - stop_loss)
        shares = min(shares, capital * leverage / entry_price)  # Adjust for leverage

        profit_target = entry_price + direction * profit_multiplier * abs(entry_price - stop_loss)
        place_trade(entry_price, stop_loss, profit_target,direction)
        if day_data['date'].loc[-1].dt.time > target_time:
          # Close all positions
          trades = []
          positions = ib.positions()
          for position in positions:
              trade = close_position(position)
              trades.append(trade)

          # Monitor the status of the trades
          for trade in trades:
              while trade.orderStatus.status in ['PendingSubmit', 'Submitted']:
                  print(f'Order Status: {trade.orderStatus.status}')
                  ib.sleep(5)  # Wait 5 seconds before checking again
              print(f'Final Order Status: {trade.orderStatus.status}')
    return None


IndentationError: expected an indented block (3351827069.py, line 100)

In [47]:
ib.connect('127.0.0.1',7497,clientId=10)
contract = Stock('QQQ','SMART','USD')
# Send the order
# trade = ib.placeOrder(contract, order)
# while trade.orderStatus.status in ['PendingSubmit', 'Submitted']:
#     print(f'Order Status: {trade.orderStatus.status}')
#     time.sleep(5)  # Wait 5 seconds before checking again

# print(f'Final Order Status: {trade.orderStatus.status}')
order = MarketOrder('BUY', 1)
trade = ib.placeOrder(contract, order)
# Set stop loss and profit target
stop_loss_order = StopOrder('SELL', 1, 472.44)
profit_target_order = LimitOrder('SELL', 1, 500)


In [50]:
ib.connect('127.0.0.1',7497,clientId=10)
contract = Stock('QQQ','SMART','USD')

ib.placeOrder(contract, stop_loss_order)
ib.placeOrder(contract, profit_target_order)


Trade(contract=Stock(symbol='QQQ', exchange='SMART', currency='USD'), order=LimitOrder(orderId=7, clientId=10, action='SELL', totalQuantity=1, lmtPrice=500), orderStatus=OrderStatus(orderId=7, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 12, 20, 52, 44, 60057, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

In [52]:
open_orders = ib.openOrders()
print(open_orders)
# Display pending orders
# for order in open_orders:
#     print(f"Order ID: {order.order.orderId}")
#     print(f"Action: {order.order.action}")
#     print(f"Quantity: {order.order.totalQuantity}")
#     print(f"Type: {order.order.orderType}")
#     print(f"Status: {order.orderStatus.status}")
#     print(f"Symbol: {order.contract.symbol}")
#     print(f"Price: {order.order.lmtPrice if order.order.orderType == 'LMT' else 'N/A'}")
#     print(f"Stop Price: {order.order.auxPrice if order.order.orderType == 'STP' else 'N/A'}")
#     print(f"Time in Force: {order.order.timeInForce}")
#     print('-' * 30)

# Disconnect from IBKR TWS
ib.disconnect()

[StopOrder(orderId=6, clientId=10, permId=581124642, action='SELL', totalQuantity=1.0, lmtPrice=0.0, auxPrice=472.44), LimitOrder(orderId=7, clientId=10, permId=581124643, action='SELL', totalQuantity=1.0, lmtPrice=500.0, auxPrice=0.0)]


In [40]:
ib = IB()
ib.connect('127.0.0.1',7497,clientId=110)
order = MarketOrder('SELL', 5)
trade = ib.placeOrder(contract, order)


In [38]:
positions = ib.positions()
positions_df = util.df(positions)
print(positions_df)

     account                                           contract  position  \
0  DUA169396  Stock(conId=320227571, symbol='QQQ', exchange=...       5.0   

      avgCost  
0  473.550052  


In [37]:
import nest_asyncio
nest_asyncio.apply()
ib = IB()
ib.connect('127.0.0.1',7497,clientId=110)
positions = ib.positions()
ib.qualifyContracts(contract)

def close_position(position):
    contract = Stock('QQQ','SMART','USD')
    try:
        contract = position.contract
        quantity = position.position
        if quantity > 0:
            order = MarketOrder('SELL', quantity)
        else:
            order = MarketOrder('BUY', -quantity)
        trade = ib.placeOrder(contract, order)
        return trade
    except Exception as e:
        print(f"Error:{e}")
# Close all positions
trades = []
for position in positions:
    trade = close_position(position)
    trades.append(trade)

# Monitor the status of the trades
for trade in trades:
    while trade.orderStatus.status in ['PendingSubmit', 'Submitted']:
        print(f'Order Status: {trade.orderStatus.status}')
        ib.sleep(5)  # Wait 5 seconds before checking again
    print(f'Final Order Status: {trade.orderStatus.status}')

# Disconnect from IBKR TWS
ib.disconnect()

Error 200, reqId 18: No security definition has been found for the request
Canceled order: Trade(contract=Stock(conId=320227571, symbol='QQQ', exchange='NASDAQ', currency='USD', localSymbol='QQQ', tradingClass='NMS'), order=MarketOrder(orderId=18, clientId=110, action='SELL', totalQuantity=5.0), orderStatus=OrderStatus(orderId=18, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 12, 20, 37, 29, 735942, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 12, 20, 37, 29, 777062, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 200, reqId 18: No security definition has been found for the request', errorCode=200)], advancedError='')


Order Status: PendingSubmit
Final Order Status: Cancelled
